# 🌽 Entrenamiento MobileNetV3 - V4 FINAL (A100 OPTIMIZADO)

**Objetivo: >85% Accuracy + >80% Recall en TODAS las clases**

## 🎯 Estrategia V4 FINAL:
1. ✅ **Batch size 32** (probado en V2 - TODAS las clases >80% recall)
2. ✅ **100 épocas** (vs 80 en V2 - mayor convergencia)
3. ✅ **Mixed Precision FP16** (A100 Tensor Cores - 2x velocidad)
4. ✅ **Categorical crossentropy** (ya funciona - no cambiar)
5. ✅ **Sin fine-tuning** (evita colapso)
6. ✅ **Arquitectura 384→192** (probada)

## 📊 Análisis de resultados previos:

### V2 (80 épocas, batch 32) - ✅ CONFIGURACIÓN GANADORA:
- Test Accuracy: 84.53%
- **Gray_Leaf_Spot recall: >80%** ✅
- TODAS las clases >80% recall ✅

### V3.1 SAFE (100 épocas, batch 64) - ❌ BATCH 64 FALLA:
- Test Accuracy: 84.85% (mejor)
- **Gray_Leaf_Spot recall: 76.60%** ❌ (colapso)
- Blight: 86.18% ✅
- Common_Rust: 87.71% ✅
- Healthy: 88.89% ✅

## 🔍 Conclusión:
**Batch size 64 perjudica el aprendizaje de Gray_Leaf_Spot**

**Solución: Volver a batch 32 (V2) + aumentar a 100 épocas + A100 FP16**

---

## 🔧 BLOQUE 1: Setup y Verificación

In [ ]:
# 1.1 Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 1.2 Clonar repositorio
!git clone -b main https://github.com/ojgonzalezz/corn-diseases-detection.git
%cd corn-diseases-detection/entrenamiento_modelos

# 1.3 Instalar dependencias
!pip install -q -r requirements.txt

# 1.4 Crear directorios necesarios en Drive
!mkdir -p /content/drive/MyDrive/corn-diseases-detection/models
!mkdir -p /content/drive/MyDrive/corn-diseases-detection/logs
!mkdir -p /content/drive/MyDrive/corn-diseases-detection/mlruns

print("\n✅ Setup completado!")

## ⚡ BLOQUE 2: Activar Mixed Precision (A100 Tensor Cores)

In [ ]:
import tensorflow as tf
from tensorflow.keras import mixed_precision

# Activar mixed precision para A100 (usa Tensor Cores)
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

print(f"\n{'='*60}")
print("⚡ MIXED PRECISION ACTIVADO (A100 TENSOR CORES)")
print(f"{'='*60}")
print(f"Compute dtype: {policy.compute_dtype}")
print(f"Variable dtype: {policy.variable_dtype}")
print(f"\n✅ Velocidad esperada: 2x vs FP32")
print(f"✅ Accuracy degradación: <0.1%")
print(f"✅ VRAM ahorro: ~40%")
print(f"{'='*60}\n")

## 🏗️ BLOQUE 3: Configuración y Generadores (BATCH 32)

In [ ]:
import os
import time
import numpy as np
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers.schedules import CosineDecay
from sklearn.utils.class_weight import compute_class_weight

# Importar configuración base
from config import *
from utils import setup_gpu

# ==================== CONFIGURACIÓN V4 FINAL ====================
BATCH_SIZE = 32  # PROBADO en V2 - TODAS las clases >80% recall
EPOCHS = 120  # Aumentado de 80 (V2) a 120 para mejor convergencia
LEARNING_RATE = 0.001  # LR inicial
EARLY_STOPPING_PATIENCE = 30  # Paciencia para 100 épocas

# Configurar GPU
setup_gpu(GPU_MEMORY_LIMIT)

print(f"\n{'='*60}")
print("🚀 CONFIGURACIÓN V4 FINAL - A100 OPTIMIZADO")
print(f"{'='*60}")
print(f"Batch Size: {BATCH_SIZE} (probado en V2 ✅)")
print(f"Épocas: {EPOCHS} (vs 80 en V2)")
print(f"Learning Rate: {LEARNING_RATE} (Cosine Decay)")
print(f"Loss: Categorical Crossentropy (ya funciona)")
print(f"Mixed Precision: ACTIVADO (FP16)")
print(f"Fine-tuning: DESHABILITADO")
print(f"\n⏱️ Tiempo estimado: 140-160 min (vs 287 min en V3.1 FP32)")
print(f"📊 Accuracy esperado: >85%")
print(f"📊 Gray_Leaf_Spot recall esperado: >80% (como en V2)")
print(f"{'='*60}\n")

In [ ]:
# Crear generadores de datos con BATCH SIZE 32
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("Creando generadores de datos (batch 32)...\n")

# Solo rescale (augmentation ya aplicado en preprocessing)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=VAL_SPLIT + TEST_SPLIT
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=VAL_SPLIT + TEST_SPLIT
)

train_gen = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=RANDOM_SEED
)

val_gen = val_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=RANDOM_SEED
)

test_gen = val_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=RANDOM_SEED
)

print(f"📊 Dataset:")
print(f"  Training:   {train_gen.samples} imágenes ({train_gen.samples // BATCH_SIZE} batches)")
print(f"  Validation: {val_gen.samples} imágenes ({val_gen.samples // BATCH_SIZE} batches)")
print(f"  Test:       {test_gen.samples} imágenes ({test_gen.samples // BATCH_SIZE} batches)")
print(f"\n✅ Batch size 32: Configuración probada en V2")

# Calcular class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_gen.classes),
    y=train_gen.classes
)
class_weight_dict = dict(enumerate(class_weights))
print(f"\n⚖️ Class weights: {class_weight_dict}")

## 🏗️ BLOQUE 4: Crear Modelo (Arquitectura V2 probada)

In [ ]:
# Crear modelo V4 FINAL (misma arquitectura que V2)
def create_v4_final_model(num_classes, image_size, initial_learning_rate, steps_per_epoch, total_epochs):
    """
    Arquitectura V4 FINAL - Batch 32 + 100 épocas + Mixed Precision
    
    Configuración probada en V2:
    - Dense(384) → Dense(192): ✅ TODAS las clases >80% recall
    - Dropout(0.4, 0.35): ✅ Regularización óptima
    - Batch 32: ✅ Gray_Leaf_Spot aprendido correctamente
    - Categorical crossentropy: ✅ Ya funciona
    
    Mejoras vs V2:
    - 100 épocas (vs 80): Mayor convergencia
    - Mixed precision FP16: 2x velocidad en A100
    """
    
    # Cargar base preentrenada
    base_model = MobileNetV3Large(
        input_shape=(*image_size, 3),
        include_top=False,
        weights='imagenet'
    )
    
    # Congelar TODAS las capas base (NO fine-tuning)
    base_model.trainable = False
    
    # ARQUITECTURA 384 → 192 (probada en V2)
    inputs = tf.keras.Input(shape=(*image_size, 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    
    # Primera capa densa: 384 neuronas
    x = Dense(384, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = Dropout(0.4)(x)
    
    # Segunda capa densa: 192 neuronas
    x = Dense(192, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = Dropout(0.35)(x)
    
    # Output layer (FP32 para estabilidad numérica)
    outputs = Dense(num_classes, activation='softmax', dtype='float32')(x)
    
    model = Model(inputs, outputs)
    
    # Cosine Decay ajustado a 100 épocas
    lr_schedule = CosineDecay(
        initial_learning_rate=initial_learning_rate,
        decay_steps=steps_per_epoch * total_epochs,
        alpha=0.1  # LR final = 10% del inicial
    )
    
    # Compilar con categorical crossentropy (ya funciona)
    model.compile(
        optimizer=Adam(learning_rate=lr_schedule),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Crear modelo
print("\n🏗️ Creando modelo V4 FINAL (arquitectura V2 probada)...\n")
steps_per_epoch = train_gen.samples // BATCH_SIZE

model = create_v4_final_model(
    num_classes=NUM_CLASSES,
    image_size=IMAGE_SIZE,
    initial_learning_rate=LEARNING_RATE,
    steps_per_epoch=steps_per_epoch,
    total_epochs=EPOCHS
)

print(f"📐 Total parámetros: {model.count_params():,}")
trainable_params = sum([tf.size(w).numpy() for w in model.trainable_weights])
print(f"📐 Parámetros entrenables: {trainable_params:,}")
print(f"📐 Ratio datos/params: {train_gen.samples / trainable_params:.2f}")

print(f"\n⚡ Configuración V4 FINAL:")
print(f"   • Arquitectura: Dense(384)→Dense(192) (V2 probada ✅)")
print(f"   • Batch 32: Probado en V2 con >80% recall en todas las clases ✅")
print(f"   • 100 épocas: Mayor convergencia vs 80 (V2)")
print(f"   • Mixed precision: {policy.compute_dtype} compute, {policy.variable_dtype} variables")
print(f"   • Loss: Categorical crossentropy (ya funciona ✅)")
print(f"   • Sin fine-tuning: Evita colapso ✅")

print("\n✅ Modelo V4 FINAL creado!")

## 🚀 BLOQUE 5: Entrenamiento (100 épocas)

In [ ]:
# Callbacks optimizados
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=EARLY_STOPPING_PATIENCE,
        restore_best_weights=True,
        verbose=1,
        mode='max'
    ),
    ModelCheckpoint(
        str(MODELS_DIR / 'mobilenetv3_v4_final_best.keras'),
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1,
        mode='max'
    )
]

print(f"\n{'='*60}")
print("🚀 INICIANDO ENTRENAMIENTO V4 FINAL")
print(f"{'='*60}\n")
print("🎯 Objetivo: >85% accuracy, >80% recall en TODAS las clases")
print("\n📊 Estrategia:")
print(f"   • Batch 32: Configuración probada en V2 ✅")
print(f"   • 100 épocas: Más convergencia que V2 (80 épocas)")
print(f"   • Mixed precision FP16: 2x velocidad en A100")
print(f"   • Categorical crossentropy: Ya funciona correctamente")
print(f"   • Sin fine-tuning: Evita colapso")
print(f"\n📊 Resultados previos:")
print(f"   • V2 (80 épocas, batch 32):    84.53% - ✅ TODAS >80% recall")
print(f"   • V3.1 (100 épocas, batch 64): 84.85% - ❌ Gray_Leaf_Spot 76.60%")
print(f"   • V4 FINAL (100 épocas, batch 32 + FP16): Esperado >85% + todas >80%")
print(f"\n⏱️ Tiempo estimado: 140-160 min (2x más rápido que V3.1 FP32)")
print(f"{'='*60}\n")

start_time = time.time()

history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen,
    callbacks=callbacks,
    class_weight=class_weight_dict,
    verbose=1
)

training_time = time.time() - start_time
best_val_acc = max(history.history['val_accuracy'])
best_epoch = history.history['val_accuracy'].index(best_val_acc) + 1

print(f"\n{'='*60}")
print("✅ ENTRENAMIENTO V4 FINAL COMPLETADO")
print(f"{'='*60}")
print(f"⏱️  Tiempo: {training_time/60:.2f} minutos")
print(f"⚡ Speedup vs V3.1 FP32: {287.78/(training_time/60):.2f}x más rápido")
print(f"📊 Mejor Val Accuracy: {best_val_acc:.4f} ({best_val_acc*100:.2f}%) en época {best_epoch}")
print(f"📊 Train Accuracy final: {history.history['accuracy'][-1]:.4f}")

if best_val_acc >= 0.85:
    print(f"\n🎉 ¡OBJETIVO DE ACCURACY ALCANZADO! (>85%)")
    improvement = (best_val_acc - 0.8485) * 100
    print(f"📈 Mejora vs V3.1: +{improvement:.2f} puntos porcentuales")
else:
    gap = (0.85 - best_val_acc) * 100
    print(f"\n⚠️  Faltaron {gap:.2f} puntos porcentuales para 85%")

print(f"{'='*60}\n")

## 📊 BLOQUE 6: Evaluación Detallada y Guardado

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import json
from datetime import datetime
from utils import evaluate_model, plot_training_history, plot_confusion_matrix, save_training_log

print(f"\n{'='*60}")
print("📊 EVALUACIÓN EN TEST SET")
print(f"{'='*60}\n")

# Evaluar modelo en test set
evaluation_results = evaluate_model(model, test_gen, CLASSES)

test_acc = evaluation_results['test_accuracy']
test_loss = evaluation_results['test_loss']

print(f"\n{'='*60}")
print("📈 RESULTADOS FINALES V4 FINAL")
print(f"{'='*60}")
print(f"Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"Test Loss:     {test_loss:.4f}")

# Comparación con versiones anteriores
v2_acc = 0.8453
v31_acc = 0.8485
improvement_v2 = (test_acc - v2_acc) * 100
improvement_v31 = (test_acc - v31_acc) * 100

print(f"\n📊 Comparación:")
print(f"   V2 (80 épocas, batch 32):     {v2_acc*100:.2f}%")
print(f"   V3.1 (100 épocas, batch 64):  {v31_acc*100:.2f}%")
print(f"   V4 FINAL (100 épocas, batch 32 + FP16): {test_acc*100:.2f}%")
print(f"   Mejora vs V2:     {improvement_v2:+.2f} puntos")
print(f"   Mejora vs V3.1:   {improvement_v31:+.2f} puntos")

# Verificar objetivo de accuracy
if test_acc >= 0.85:
    print(f"\n🎉 ¡OBJETIVO DE ACCURACY ALCANZADO! (>85%)")
else:
    print(f"\n⚠️  Accuracy: {test_acc:.4f} vs objetivo 0.85")

print(f"\n{'='*60}")
print("📋 MÉTRICAS POR CLASE")
print(f"{'='*60}")

recall_objetivo_alcanzado = True
gray_leaf_spot_recall = 0.0

for class_name in CLASSES:
    metrics = evaluation_results['classification_report'][class_name]
    recall = metrics['recall']
    precision = metrics['precision']
    f1 = metrics['f1-score']
    
    # Guardar recall de Gray_Leaf_Spot
    if 'Gray' in class_name or 'gray' in class_name.lower():
        gray_leaf_spot_recall = recall
    
    status = "✅" if recall >= 0.80 else "❌"
    
    print(f"\n{status} {class_name}:")
    print(f"  Precision: {precision:.4f} ({precision*100:.2f}%)")
    print(f"  Recall:    {recall:.4f} ({recall*100:.2f}%)")
    print(f"  F1-Score:  {f1:.4f} ({f1*100:.2f}%)")
    
    if recall < 0.80:
        recall_objetivo_alcanzado = False

# Verificar objetivo de recall
if recall_objetivo_alcanzado:
    print(f"\n🎉 ¡OBJETIVO DE RECALL ALCANZADO EN TODAS LAS CLASES! (>80%)")
    if gray_leaf_spot_recall > 0:
        improvement_gls = (gray_leaf_spot_recall - 0.7660) * 100
        print(f"\n🌟 Gray_Leaf_Spot: {gray_leaf_spot_recall*100:.2f}% (mejora de {improvement_gls:+.2f} puntos vs V3.1)")
        print(f"✅ Batch 32 funciona correctamente para Gray_Leaf_Spot (como en V2)")
else:
    print(f"\n⚠️  Algunas clases tienen recall < 80%")

# Resumen final
print(f"\n{'='*60}")
print("🎯 VERIFICACIÓN DE OBJETIVOS")
print(f"{'='*60}")
print(f"✓ Accuracy >85%: {'✅ SÍ' if test_acc >= 0.85 else '❌ NO'} ({test_acc*100:.2f}%)")
print(f"✓ Recall >80%:   {'✅ SÍ' if recall_objetivo_alcanzado else '❌ NO'} (todas las clases)")

if test_acc >= 0.85 and recall_objetivo_alcanzado:
    print(f"\n🏆 ¡AMBOS OBJETIVOS ALCANZADOS!")
    print(f"\n✅ Estrategia correcta:")
    print(f"   • Batch 32 (probado en V2)")
    print(f"   • 100 épocas (mayor convergencia)")
    print(f"   • Mixed precision FP16 (2x velocidad)")
    print(f"   • Categorical crossentropy (ya funciona)")

print(f"{'='*60}\n")

## 💾 BLOQUE 7: Guardar Resultados

In [ ]:
# Guardar resultados
print("💾 Guardando resultados V4 FINAL...\n")

# 1. Gráfico de entrenamiento
plot_path = LOGS_DIR / 'mobilenetv3_v4_final_training_history.png'
plot_training_history(history, plot_path)
print(f"✅ Gráfico guardado: {plot_path}")

# 2. Matriz de confusión
cm_path = LOGS_DIR / 'mobilenetv3_v4_final_confusion_matrix.png'
cm = plot_confusion_matrix(
    evaluation_results['y_true'],
    evaluation_results['y_pred'],
    CLASSES,
    cm_path
)
print(f"✅ Matriz de confusión guardada: {cm_path}")

# 3. Modelo final
model_path = MODELS_DIR / 'mobilenetv3_v4_final.keras'
model.save(str(model_path))
print(f"✅ Modelo final guardado: {model_path}")

# 4. Log detallado
hyperparameters = {
    'model_name': 'MobileNetV3-Large V4 FINAL',
    'version': 'V4 FINAL - Batch 32 + 120 épocas + A100 FP16',
    'architecture': 'Dense(384)->Dense(192)',
    'image_size': IMAGE_SIZE,
    'batch_size': BATCH_SIZE,
    'epochs': EPOCHS,
    'learning_rate': LEARNING_RATE,
    'lr_schedule': 'CosineDecay',
    'optimizer': 'Adam',
    'loss_function': 'categorical_crossentropy',
    'dropout': [0.4, 0.35],
    'l2_regularization': 0.001,
    'mixed_precision': 'mixed_float16',
    'fine_tuning': 'Disabled',
    'gpu_optimization': 'A100 24GB with Tensor Cores',
    'strategy': 'Batch 32 from V2 (proven) + 120 epochs (better convergence) + FP16 (2x speed)'
}

log_path = LOGS_DIR / 'mobilenetv3_v4_final_training_log.json'

save_training_log(
    log_path,
    'MobileNetV3-Large V4 FINAL',
    hyperparameters,
    history,
    evaluation_results,
    cm,
    training_time
)
print(f"✅ Log guardado: {log_path}")

# 5. Resumen final comparativo
print(f"\n{'='*60}")
print("🎉 ¡ENTRENAMIENTO V4 FINAL COMPLETADO!")
print(f"{'='*60}")

print(f"\n⏱️  Tiempos de entrenamiento:")
print(f"   • V2 (80 épocas, batch 32, FP32):  146.52 min")
print(f"   • V3.1 (100 épocas, batch 64, FP32): 287.78 min")
print(f"   • V4 FINAL (100 épocas, batch 32, FP16): {training_time/60:.2f} min")
print(f"   • Speedup vs V2:   {146.52/(training_time/60):.2f}x")
print(f"   • Speedup vs V3.1: {287.78/(training_time/60):.2f}x")

print(f"\n📊 Test Accuracy:")
print(f"   • V2 (80 épocas, batch 32):      84.53%")
print(f"   • V3.1 (100 épocas, batch 64):   84.85%")
print(f"   • V4 FINAL (100 épocas, batch 32 + FP16): {test_acc*100:.2f}%")

print(f"\n📋 Gray_Leaf_Spot Recall (problema clave):")
print(f"   • V2 (batch 32):  >80% ✅")
print(f"   • V3.1 (batch 64): 76.60% ❌")
if gray_leaf_spot_recall > 0:
    print(f"   • V4 FINAL (batch 32): {gray_leaf_spot_recall*100:.2f}% {'✅' if gray_leaf_spot_recall >= 0.80 else '❌'}")

print(f"\n🎯 Objetivos:")
print(f"   • Accuracy >85%: {'✅ ALCANZADO' if test_acc >= 0.85 else '❌ NO ALCANZADO'}")
print(f"   • Recall >80%:   {'✅ ALCANZADO' if recall_objetivo_alcanzado else '❌ NO ALCANZADO'}")

print(f"\n💾 Archivos guardados en:")
print(f"   • Modelo: {model_path}")
print(f"   • Logs: {LOGS_DIR}")

print(f"\n✅ Estrategia V4 FINAL:")
print(f"   • Batch 32: Configuración probada en V2 (>80% recall todas las clases)")
print(f"   • 100 épocas: Mayor convergencia vs V2 (80 épocas)")
print(f"   • Mixed precision FP16: 2x velocidad en A100 Tensor Cores")
print(f"   • Categorical crossentropy: Ya funciona correctamente")
print(f"   • Sin fine-tuning: Estabilidad garantizada")

print(f"\n🔍 Conclusión: Batch size 64 perjudicaba Gray_Leaf_Spot")
print(f"✅ Solución: Volver a batch 32 + aumentar épocas + FP16 para velocidad")
print(f"{'='*60}\n")